# How-To

instructions coming soon

In [ ]:
#@title Setup the app
# install packages
!pip install tensorflow
!pip install opencv-python
!pip install tensorflow_hub
!pip install matplotlib
!pip install numpy
!pip install Pillow

# download tf model
!curl -L https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2?tf-hub-format=compressed --output model.tar.gz
!mkdir tf_model
!tar -xf model.tar.gz -C tf_model/

# clone project repo from github
!mkdir repo
!git clone https://github.com/maxupravitelev/neural-style-transfer-video.git ./repo
!cp -R repo/modules modules

The output video files have a rather low resolution, due to how the actual neural style transfer is handled. You can upscale the video files either by simple upscaling or by utilizing the [Image Super Resolution](https://github.com/idealo/image-super-resolution) package. If you want to use the latter, please run the cell below and click on "Restart Runtime" once it's done.

In [ ]:
#@title Optional: Install the Image Super Resolution package
!pip install ISR
!pip install 'h5py==2.10.0' --force-reinstall

In [ ]:
#@title Run the app
from modules.FrameExtractor import FrameExtractor
from modules.NeuralStyleTransferHandler import NeuralStyleTransferHandler
from modules.ImageResizer import ImageResizer
from modules.FrameCombiner import FrameCombiner

import logging
import os

logging.basicConfig(level=logging.DEBUG, datefmt='%Y-%m-%d:%H:%M:%S', format='%(asctime)s | %(levelname)s | %(name)s | LINE %(lineno)d: %(message)s')
log = logging.getLogger(__name__)
log.debug('Logging initialized')


input_file_path = "testvid.mp4" #@param {type:"string"}
filter_path = "cam0.jpg" #@param {type:"string"}
output_max_size = 512 #@param {type:"number"}
resize_factor = 2 #@param {type:"number"}
use_superres = True #@param {type:"boolean"}


def main():

    # extract frames in folder
    extracted_frames_path, input_video_filename = FrameExtractor(input_file_path).extract_frames()

    # nst all frames in folder
    styled_frames_path = NeuralStyleTransferHandler(extracted_frames_path, input_video_filename, output_max_size).stylize_batch('stylize_by_filter', filter_path)

    # optional: resize all frames in folder
    if resize_factor is not 1:
        if use_superres is True:
            resized_frames_path = ImageResizer(styled_frames_path, input_video_filename).resize_superres(resize_factor)
            styled_frames_path = resized_frames_path
        else:
            resized_frames_path = ImageResizer(styled_frames_path, input_video_filename).resize(resize_factor)
            styled_frames_path = resized_frames_path


    # combine all frames in video file
    output_video_path = FrameCombiner(styled_frames_path, input_video_filename).combine_frames(filter_path)

main()


In [ ]:
#@title Download all output as a zip file
!zip -r ./output.zip ./output

from google.colab import files
files.download("output.zip")

In [ ]:
#@title Remove output folder
%rm -r output